In [2]:
!pip3 install folium

     |████████████████████████████████| 95 kB 1.4 MB/s eta 0:00:01
You should consider upgrading via the '/home/leon/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
import uploader
import psycopg2
import folium
import pandas as pd

In [5]:
loader = uploader.TrafficDatabaseLoader('psql_config.json')
loader.import_from_osm('/home/leon/Downloads/map.osm')

/home/leon/anaconda3/lib/python3.8/site-packages/sqlalchemy/ext/declarative/clsregistry.py:125: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.transport_type, and will be replaced in the string-lookup table.
  util.warn(
/home/leon/anaconda3/lib/python3.8/site-packages/sqlalchemy/ext/declarative/clsregistry.py:125: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.transport_model, and will be replaced in the string-lookup table.
  util.warn(
/home/leon/anaconda3/lib/python3.8/site-packages/sqlalchemy/ext/declarative/clsregistry.py:125: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.relation, and will be replaced in the string-lookup table.
  util.warn(
/home/leon/anaconda3/lib/python3.8/site-packages/sqlalchemy/ext/declarative/clsregistry.py:125: SAWarning: T

In [6]:
conn = psycopg2.connect(dbname='traffic', user='postgres', 
                        password='smarttraffic', host='localhost')

cursor = conn.cursor()

cursor.execute("SELECT * FROM Node WHERE id in (SELECT node_id FROM Node_tag WHERE k LIKE 'highway')")
records = cursor.fetchall()
df = pd.DataFrame(records)

cursor.close()

In [7]:
df.rename(columns={0:'id', 1:'lat', 2:'lon'}, inplace=True)
df.sample(10)

,id,lat,lon
100,7869915099,55.730838,37.623990
79,5159681564,55.730016,37.625660
28,1732706942,55.727402,37.619907
97,6438277117,55.729347,37.630080
76,5133615311,55.729295,37.619896
23,1253885411,55.732869,37.624290
65,3576624537,55.734424,37.624342
25,1670392327,55.729395,37.623567
5,250638698,55.729794,37.625842
22,1253885348,55.732648,37.624281


In [8]:
def show_circles_on_map(data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())
    m = folium.Map(location=location, zoom_start=99)

    for _, row in data.iterrows():
        folium.Circle(
            radius=1,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True
        ).add_to(m)

    return m

In [9]:
show_circles_on_map(df, 'lat', 'lon', 'blue')

In [62]:
possible_tags = "'" + "','".join(['primary', 'secondary', 'primary_link', 
                                  'secondary_link', 'unclassified', 
                                  'tertiary', 'residential']) + "'" # kek

In [63]:
conn = psycopg2.connect(dbname='traffic', user='postgres', 
                        password='smarttraffic', host='localhost')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM node, way_node_m2m, way, way_tag WHERE node.id=way_node_m2m.node_id AND way_node_m2m.way_id=way.id AND way.id=way_tag.way_id AND way_tag.k='highway' AND way_tag.v in ({possible_tags}) ORDER BY way.id, way_node_m2m.sequence_id")
records = cursor.fetchall()
df = pd.DataFrame(records)
cursor.close()

In [64]:
def show_highways(data, latitude_column, longitude_column, sequence_column, way_id_column, color, m = None):
    if not m:
        location = (data[latitude_column].mean(), data[longitude_column].mean())
        m = folium.Map(location=location, zoom_start=99)

    grouped_by_ways = data.groupby([way_id_column])
    for group_name, group in grouped_by_ways:
        points = list(zip(group[latitude_column], group[longitude_column]))
        folium.PolyLine(
            points,
            color=color, 
            weight=2.5, 
            opacity=1
        ).add_to(m)

    return m

In [65]:
show_highways(df, 1, 2, 5, 4, 'red')